<a href="https://colab.research.google.com/github/narendra7959/Deep-Learning-Practicals/blob/main/deep_learning_lab_2_24040208002.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## MDSC-302 | ASSIGNMENT-2 | 24040208002

### Loading an image file

In [ ]:
pip install imageio

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import imageio.v2 as imageio
img_arr = imageio.imread(r'dlwpt-code-master\data\p1ch4\image-dog\bobby.jpg')

img_arr.shape

(720, 1280, 3)

### Changing the layout

In [ ]:
import torch
img = torch.from_numpy(img_arr)
out = img.permute(2, 0, 1)

In [ ]:
batch_size = 3
batch = torch.zeros(batch_size, 3, 256, 256, dtype=torch.uint8)

# This indicates that our batch will consist of three RGB images 256 pixels in height and 256 pixels in width


#### We can now load all PNG images from an input directory and store them in the tensor:


In [ ]:
import os
data_dir = r'D:\MDSC-302(P)\dlwpt-code-master\data\p1ch4\image-cats'
filenames = [name for name in os.listdir(data_dir)
            if os.path.splitext(name)[-1] == '.png']
for i, filename in enumerate(filenames):
    img_arr = imageio.imread(os.path.join(data_dir, filename))
    img_t = torch.from_numpy(img_arr)
    img_t = img_t.permute(2, 0, 1)
    img_t = img_t[:3]
    batch[i] = img_t

### Normalizing the Data

In [ ]:
batch = batch.float()
batch /= 255.0

#### Another possibility is to compute the mean and standard deviation of the input data and scale it so that the output has zero mean and unit standard deviation across each channel:



In [ ]:
n_channels = batch.shape[1]
for c in range(n_channels):
 mean = torch.mean(batch[:, c])
 std = torch.std(batch[:, c])
 batch[:, c] = (batch[:, c]- mean) / std


### Loading a Specialized Format

In [ ]:
import imageio
dir_path = r"D:\MDSC-302(P)\dlwpt-code-master\data\p1ch4\volumetric-dicom\2-LUNG 3.0  B70f-04083"
vol_arr = imageio.volread(dir_path, 'DICOM')
vol_arr.shape

Reading DICOM (examining files): 1/99 files (1.0%21/99 files (21.2%26/99 files (26.3%66/99 files (66.7%99/99 files (100.0%)
  Found 1 correct series.
Reading DICOM (loading data): 33/99  (33.368/99  (68.799/99  (100.0%)


(99, 512, 512)

#### the layout is different from what PyTorch expects, due to having no channel information. So we’ll have to make room for the channel dimension using unsqueeze:



In [ ]:
vol = torch.from_numpy(vol_arr).float()
vol = torch.unsqueeze(vol, 0)

vol.shape

torch.Size([1, 99, 512, 512])

### Representing Tabular Data

#### Loading a Wine Data Tensor

In [ ]:
import csv
import numpy as np
wine_path = r"D:\MDSC-302(P)\dlwpt-code-master\data\p1ch4\tabular-wine\winequality-white.csv"
wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=";",skiprows=1)
wineq_numpy

array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]],
      shape=(4898, 12), dtype=float32)

#### Here we just prescribe what the type of the 2D array should be (32-bit floating-point), the delimiter used to separate values in each row, and the fact that the first line should not be read since it contains the column names



In [ ]:
col_list = next(csv.reader(open(wine_path), delimiter=';'))
wineq_numpy.shape, col_list

((4898, 12),
 ['fixed acidity',
  'volatile acidity',
  'citric acid',
  'residual sugar',
  'chlorides',
  'free sulfur dioxide',
  'total sulfur dioxide',
  'density',
  'pH',
  'sulphates',
  'alcohol',
  'quality'])

In [ ]:
wineq = torch.from_numpy(wineq_numpy)
wineq.shape, wineq.dtype

(torch.Size([4898, 12]), torch.float32)

### Representing Scores

In [ ]:
data = wineq[:, :-1]
data, data.shape

(tensor([[ 7.0000,  0.2700,  0.3600,  ...,  3.0000,  0.4500,  8.8000],
         [ 6.3000,  0.3000,  0.3400,  ...,  3.3000,  0.4900,  9.5000],
         [ 8.1000,  0.2800,  0.4000,  ...,  3.2600,  0.4400, 10.1000],
         ...,
         [ 6.5000,  0.2400,  0.1900,  ...,  2.9900,  0.4600,  9.4000],
         [ 5.5000,  0.2900,  0.3000,  ...,  3.3400,  0.3800, 12.8000],
         [ 6.0000,  0.2100,  0.3800,  ...,  3.2600,  0.3200, 11.8000]]),
 torch.Size([4898, 11]))

In [ ]:
target = wineq[:,-1]
target, target.shape

(tensor([6., 6., 6.,  ..., 6., 7., 6.]), torch.Size([4898]))

#### If we want to transform the target tensor in a tensor of labels, we have two options, depending on the strategy or what we use the categorical data for. One is simply to treat labels as an integer vector of scores:



In [ ]:
target = wineq[:,-1].long()
target

tensor([6, 6, 6,  ..., 6, 7, 6])

#### One hot encoding

In [ ]:
target_onehot = torch.zeros(target.shape[0], 10)
target_onehot.scatter_(1, target.unsqueeze(1), 1.0)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [ ]:
target_unsqueezed = target.unsqueeze(1)
target_unsqueezed

tensor([[6],
        [6],
        [6],
        ...,
        [6],
        [7],
        [6]])

#### when to categerize

In [ ]:
data_mean = torch.mean(data, dim=0)
data_mean

tensor([6.8548e+00, 2.7824e-01, 3.3419e-01, 6.3914e+00, 4.5772e-02, 3.5308e+01,
        1.3836e+02, 9.9403e-01, 3.1883e+00, 4.8985e-01, 1.0514e+01])

In [ ]:
data_var = torch.var(data, dim=0)
data_var

tensor([7.1211e-01, 1.0160e-02, 1.4646e-02, 2.5726e+01, 4.7733e-04, 2.8924e+02,
        1.8061e+03, 8.9455e-06, 2.2801e-02, 1.3025e-02, 1.5144e+00])

In [ ]:
data_normalized = (data- data_mean) / torch.sqrt(data_var)
data_normalized

tensor([[ 1.7208e-01, -8.1761e-02,  2.1326e-01,  ..., -1.2468e+00,
         -3.4915e-01, -1.3930e+00],
        [-6.5743e-01,  2.1587e-01,  4.7996e-02,  ...,  7.3995e-01,
          1.3422e-03, -8.2419e-01],
        [ 1.4756e+00,  1.7450e-02,  5.4378e-01,  ...,  4.7505e-01,
         -4.3677e-01, -3.3663e-01],
        ...,
        [-4.2043e-01, -3.7940e-01, -1.1915e+00,  ..., -1.3130e+00,
         -2.6153e-01, -9.0545e-01],
        [-1.6054e+00,  1.1666e-01, -2.8253e-01,  ...,  1.0049e+00,
         -9.6251e-01,  1.8574e+00],
        [-1.0129e+00, -6.7703e-01,  3.7852e-01,  ...,  4.7505e-01,
         -1.4882e+00,  1.0448e+00]])

#### finding thresholds

PyTorch also provides comparison functions,
here torch.le(target, 3), but using operators
seems to be a good standard.

In [ ]:
bad_indexes = target <= 3
bad_indexes.shape, bad_indexes.dtype, bad_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(20))

The bad_indexes tensor has the same shape
as target, with values of False or True depending on the outcome of the comparison
between our threshold and each element in the original target tensor:

In [ ]:
bad_data = data[bad_indexes]
bad_data.shape

torch.Size([20, 11])

For Boolean NumPy arrays and PyTorch tensors, the & operator does a logical “and” operation.

In [ ]:
bad_data = data[target <= 3]
mid_data = data[(target > 3) & (target < 7)]
good_data = data[target >= 7]

bad_mean = torch.mean(bad_data, dim=0)
mid_mean = torch.mean(mid_data, dim=0)
good_mean = torch.mean(good_data, dim=0)

for i, args in enumerate(zip(col_list, bad_mean, mid_mean, good_mean)):
    print('{:2} {:20} {:6.2f} {:6.2f} {:6.2f}'.format(i, *args))

 0 fixed acidity          7.60   6.89   6.73
 1 volatile acidity       0.33   0.28   0.27
 2 citric acid            0.34   0.34   0.33
 3 residual sugar         6.39   6.71   5.26
 4 chlorides              0.05   0.05   0.04
 5 free sulfur dioxide   53.33  35.42  34.55
 6 total sulfur dioxide 170.60 141.83 125.25
 7 density                0.99   0.99   0.99
 8 pH                     3.19   3.18   3.22
 9 sulphates              0.47   0.49   0.50
10 alcohol               10.34  10.26  11.42


Let’s get the indexes where the total sulfur dioxide column is below the midpoint we
calculated earlier, like so:

In [ ]:
total_sulfur_threshold = 141.83
total_sulfur_data = data[:,6]
predicted_indexes = torch.lt(total_sulfur_data, total_sulfur_threshold)

predicted_indexes.shape, predicted_indexes.dtype, predicted_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(2727))

This means our threshold implies that just over half of all the wines are going to be
high quality. Next, we’ll need to get the indexes of the actually good wines:

In [ ]:
actual_indexes = target > 5
actual_indexes.shape, actual_indexes.dtype, actual_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(3258))

We will perform a logical “and” between our
prediction indexes and the actual good indexes (remember that each is just an array
of zeros and ones) and use that intersection of wines-in-agreement to determine how
well we did:

In [ ]:
n_matches = torch.sum(actual_indexes & predicted_indexes).item()
n_predicted = torch.sum(predicted_indexes).item()
n_actual = torch.sum(actual_indexes).item()
n_matches, n_matches / n_predicted, n_matches / n_actual

(2018, 0.74000733406674, 0.6193984039287906)

### working with time series

#### adding a time dimension

In [ ]:
bikes_numpy = np.loadtxt(
r"D:\MDSC-302(P)\dlwpt-code-master\data\p1ch4\bike-sharing-dataset\hour-fixed.csv",
dtype=np.float32,
delimiter=",",
skiprows=1,
converters={1: lambda x: float(x[8:10])})
bikes = torch.from_numpy(bikes_numpy)
bikes

tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 3.0000e+00, 1.3000e+01,
         1.6000e+01],
        [2.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 8.0000e+00, 3.2000e+01,
         4.0000e+01],
        [3.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 5.0000e+00, 2.7000e+01,
         3.2000e+01],
        ...,
        [1.7377e+04, 3.1000e+01, 1.0000e+00,  ..., 7.0000e+00, 8.3000e+01,
         9.0000e+01],
        [1.7378e+04, 3.1000e+01, 1.0000e+00,  ..., 1.3000e+01, 4.8000e+01,
         6.1000e+01],
        [1.7379e+04, 3.1000e+01, 1.0000e+00,  ..., 1.2000e+01, 3.7000e+01,
         4.9000e+01]])

### Shaping data by time period

We might want to break up the two-year dataset into wider observation periods, like
days. This way we’ll have N (for number of samples) collections of C sequences of length
L. In other words, our time series dataset would be a tensor of dimension 3 and shape
N × C × L. The C would remain our 17 channels, while L would be 24: 1 per hour of
the day.

In [ ]:
bikes.shape, bikes.stride()

(torch.Size([17520, 17]), (17, 1))

That’s 17,520 hours, 17 columns. Now let’s reshape the data to have 3 axes—day, hour,
and then our 17 columns:

In [ ]:
daily_bikes = bikes.view(-1, 24, bikes.shape[1])
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 24, 17]), (408, 17, 1))

We see that the rightmost dimension is the number of columns in the original
dataset. Then, in the middle dimension, we have time, split into chunks of 24 sequential
hours. In other words, we now have N sequences of L hours in a day, for C channels.
To get to our desired N × C × L ordering, we need to transpose the tensor:

In [ ]:
daily_bikes = daily_bikes.transpose(1, 2)
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 17, 24]), (408, 1, 17))

### Ready for training

In order to make it easier to render our data, we’re going to limit ourselves to the
first day for a moment. We initialize a zero-filled matrix with a number of rows equal
to the number of hours in the day and number of columns equal to the number of
weather levels:

In [ ]:
first_day = bikes[:24].long()
weather_onehot = torch.zeros(first_day.shape[0], 4)
first_day[:,9]

tensor([1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2])

Then we scatter ones into our matrix according to the corresponding level at each
row. Remember the use of unsqueeze to add a singleton dimension as we did in the
previous sections:

In [ ]:
weather_onehot.scatter_(
dim=1,
index=first_day[:,9].unsqueeze(1).long()- 1,
value=1.0)

tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]])

Last, we concatenate our matrix to our original dataset using the cat function.
Let’s look at the first of our results:

In [ ]:
torch.cat((bikes[:24], weather_onehot), 1)[:1]

tensor([[ 1.0000,  1.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  6.0000,
          0.0000,  1.0000,  0.2400,  0.2879,  0.8100,  0.0000,  3.0000, 13.0000,
         16.0000,  1.0000,  0.0000,  0.0000,  0.0000]])

We could have done the same with the reshaped daily_bikes tensor. Remember
that it is shaped (B, C, L), where L = 24. We first create the zero tensor, with the same
B and L, but with the number of additional columns as C:

In [ ]:
daily_weather_onehot = torch.zeros(daily_bikes.shape[0], 4,
daily_bikes.shape[2])
daily_weather_onehot.shape

torch.Size([730, 4, 24])

Then we scatter the one-hot encoding into the tensor in the C dimension. Since this
operation is performed in place, only the content of the tensor will change:

In [ ]:
daily_weather_onehot.scatter_(
1, daily_bikes[:,9,:].long().unsqueeze(1)- 1, 1.0)
daily_weather_onehot.shape

torch.Size([730, 4, 24])

In [ ]:
daily_bikes = torch.cat((daily_bikes, daily_weather_onehot), dim=1)

In [ ]:
daily_bikes[:, 9, :] = (daily_bikes[:, 9, :]- 1.0) / 3.0

In [ ]:
temp = daily_bikes[:, 10, :]
temp_min = torch.min(temp)
temp_max = torch.max(temp)
daily_bikes[:, 10, :] = ((daily_bikes[:, 10, :]- temp_min)
/ (temp_max- temp_min))

In [ ]:
temp = daily_bikes[:, 10, :]
daily_bikes[:, 10, :] = ((daily_bikes[:, 10, :]- torch.mean(temp))
/ torch.std(temp))

### Representing Text

#### Converting text to numbers

In [ ]:
with open(r'D:\MDSC-302(P)\dlwpt-code-master\data\p1ch4\jane-austen\1342-0.txt', encoding='utf8') as f:
 text = f.read()

##### one hot encoding characters

#### We first split our text into a list of lines and pick an arbitrary line to focus on:

In [ ]:
lines = text.split('\n')
line = lines[200]
line

'“Impossible, Mr. Bennet, impossible, when I am not acquainted with him'

#### Let’s create a tensor that can hold the total number of one-hot-encoded characters for the whole line:


In [ ]:
letter_t = torch.zeros(len(line), 128)
letter_t.shape

torch.Size([70, 128])

#### The index where the one has to be set corresponds to the index of the character in the encoding:


In [ ]:
for i, letter in enumerate(line.lower().strip()):
    letter_index = ord(letter) if ord(letter) < 128 else 0
    letter_t[i][letter_index] = 1

#### One Hot encoding whole words

#### We’ll define clean_words, which takes text and returns it in lowercase and stripped of punctuation.


In [ ]:
def clean_words(input_str):
    punctuation = '.,;:"!?”“_-'
    word_list = input_str.lower().replace('\n',' ').split()
    word_list = [word.strip(punctuation) for word in word_list]
    return word_list

words_in_line = clean_words(line)
line, words_in_line

('“Impossible, Mr. Bennet, impossible, when I am not acquainted with him',
 ['impossible',
  'mr',
  'bennet',
  'impossible',
  'when',
  'i',
  'am',
  'not',
  'acquainted',
  'with',
  'him'])

#### Next, let’s build a mapping of words to indexes in our encoding:

In [ ]:
word_list = sorted(set(clean_words(text)))
word2index_dict = {word: i for (i, word) in enumerate(word_list)}
len(word2index_dict), word2index_dict['impossible']

(7261, 3394)

####  We create an empty vector and assign the one-hot-encoded values of the word in the sentence:


In [ ]:
word_t = torch.zeros(len(words_in_line), len(word2index_dict))
for i, word in enumerate(words_in_line):
    word_index = word2index_dict[word]
    word_t[i][word_index] = 1
    print('{:2} {:4} {}'.format(i, word_index, word))

print(word_t.shape)

 0 3394 impossible
 1 4305 mr
 2  813 bennet
 3 3394 impossible
 4 7078 when
 5 3315 i
 6  415 am
 7 4436 not
 8  239 acquainted
 9 7148 with
10 3215 him
torch.Size([11, 7261])
